In [1]:
#作ったモデルを動かしてみる
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()


/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_path="../../models/hf"
model_path="../../models/hf_fin"
#model_path="../../models/hf_fin_shuffle"
#model_path="../../models/hf_10000"
model_path="../../models/hf/0312wiki300m_grad_en_to_ja_fin"
model_path="../../models/hf/0313wiki300m_grad_en_to_ja_fin_65k_vocab"
model_path="../../models/hf/0314gpt_mc4_wiki_en"
model = AutoModelForCausalLM.from_pretrained(model_path,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)


/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of te

In [4]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, max_new_tokens=200, repetition_penalty=1.5)

In [13]:
import time
text_list=["今日はいい",
"富士山は",
"質問: 今日の天気は? 回答:",
"批判的な",
"大規模言語モデルは",
"AI研究の問題点は",
"化学研究の問題点は",
"I have a",
"Functional polymers are",
"機能性高分子は",           
"ホンダ フィット 販売",
]

for text in text_list:
    perp=perplexity(model,tokenizer,text)
    s_time=time.time()
    res=pipe(text)[0]["generated_text"]
    consumed_time=time.time()-s_time
    print("-------")
    print("input: ", text)
    print("perplexity: ",perp)
    print("time: ", consumed_time)
    print("time/character: ", consumed_time/len(res))
    print("output: ",res)

/home/setup/miniconda3/envs/scr/lib/python3.11/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
input:  今日はいい
perplexity:  2121.0458984375
time:  2.01485538482666
time/character:  0.002367632649620047
output:  今日はいい天気ですね。
2016/3月5日 (水) | 固定リンク The following is a list of notable people who have been killed in the United States. One person, John "John" Johnson was born on November 7th and raised to an American family at age three or four years old when he began his career as one-time U.S. Representative from New York City's Lower East Side; another woman named Robert Astorson became active during World War II with her husband James Biggs Jr. Sr. after being arrested by President Obama for failing health reasons due to illnesses that she had died before their divorce proceedings were held there so they could not be treated properly because it did exist until June 4–8 September 9:15 pm EDT – October 1st Placement Day - January 1nd Placency & March 2rd Placer Listing Committee Report Notice Article About whether anybody else has survived this article


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
input:  富士山は
perplexity:  1864.47900390625
time:  1.4187180995941162
time/character:  0.0025379572443544117
output:  富士山は、富士山の山頂から富士山を眺めることが出来る。
富士登山口(標高1,260m)からの眺望と富士山の展望をお楽しみください!
富士山の絶景が望める富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山を望む富士山


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

-------
input:  I have a
perplexity:  276.3694152832031
time:  1.4475955963134766
time/character:  0.001434683445305725
output:  I have a 10-round, and then they are able to reach their own speeds.
The first two teams were formed in early December of that year by former head coach John McEvoy at The University of Toronto where he was named as one team's best player ever since his arrival on September 27th when it became known for its "most successful" performance with an undefeated record attendance (4–3). He finished second behind fellow Canadian Footballer Tony Bellimore who had been selected from Canada after being fired following this season due to poor results during which time MacKenzie went into retirement because there is no doubt about him leaving any club or staff members involved within them; however, some fans believe these players will be unable to compete against each other if not all games go out together again but instead play every game before returning home once more 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
input:  Functional polymers are
perplexity:  2335.94580078125
time:  1.300295114517212
time/character:  0.001138612184340816
output:  Functional polymers are the most widely used in organic chemistry.
The first step is to develop a new method of synthesising and refinement, which allows for high-quality production as well as an application that can be applied with different methods such as electrochemistry or biomedical applications (e.g.: "the effect"). The second steps were developed by Drexel et al. at Heidelberg University where he was awarded his Ph.D degree from Cornell College on behalf of MIT's Nobel Prize winning research fellowship program; however this has not been proven since it had become known today due to its lack of funding available through scientific institutions like universities around them before their creation there have already begun work into developing molecular biology rather than using other techniques including microbiological studies conducted duri

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------
input:  機能性高分子は
perplexity:  1740.2838134765625
time:  1.2984933853149414
time/character:  0.003033863049801265
output:  機能性高分子は、その構造を「高分子」と呼びます。
この構造の構造が、分子量や分子量によって決定されるので、分子量が多ければ多いほど、分子量も大きくなりやすいという特徴があります。「高分子」とは、分子量が多いことで、分子量が多くなる仕組みのことを言います(図1)。
例えば、分子量の多いことに加えて、分子量が少ないことがあげられます。(図2)、分子量が少なくなると、分子量が増えにくくなります。そのために、分子量を多くする機構が必要で、「高分子」には、分子量を増やさない機構が必要です。また、分子量を増やすために、分子量を少なくしたり、分子量を減らさなかったりすると、分子量が増えるため、分子量を減らすことができます。さらに、分子量の増加を抑えるための工夫として、分子量を増やさないようにするため、分子量を増やして、分子量を増やしていきます。
このように、分子量が増えたり減らすことにより、分子量が増加しやすくなります。分子量が増えてしまうと、分子量が減って
-------
input:  ホンダ フィット 販売
perplexity:  137.8208770751953
time:  1.3058853149414062
time/character:  0.0025555485615291708
output:  ホンダ フィット 販売
【送料無料】 【2019年モデル】【メーカー正規カスタム品、日本仕様・代引き決済不可】[Callaway/キャロウェイ]XR-685 アイアン[#3,#4][NSプロモーダス3ツアー170シャフト] スチールシャフト [Titleist]【■ACC■】 ■マルマンゴルフ(MARVEL)ドライバー (フジクラ社製)(保証書付き)[トゥルーテンパーDynamicGoldAMT](グラファイトデザイン TourAD GP-600Dカーボンシャフト)【smtb-s】(タイトリスト TITLEIST ユーティリティー ATTAS G7 IRON単品(#4～PW) N.S.PRO Regio For